In [ ]:
import pandas as pd,sqlite3

con=sqlite3.connect("FinalDB.db")
cur=con.cursor()

%load_ext sql

df1 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df2 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df3 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df1.to_sql("CENSUS_DATA", con, if_exists="replace", method="multi")
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists="replace", method="multi")
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists="replace", method="multi")

%sql sqlite:///FinalDB.db

#Find the total number of crimes recorded in the CRIME table.
%sql select name from sqlite_master where type="table"

#List community area names and numbers with per capita income less than 11000.
%sql select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME\
from CENSUS_DATA where PER_CAPITA_INCOME<11000\
order by COMMUNITY_AREA_NUMBER

#List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)
%sql select CASE_NUMBER from CHICAGO_CRIME_DATA \
where UPPER(PRIMARY_TYPE) like "%MINOR%" or\
UPPER(PRIMARY_TYPE) like "%CHILDREN%" or\
UPPER(PRIMARY_TYPE) like "%JUVENILE%" \
or UPPER(DESCRIPTION) like "%MINOR" or\
UPPER(DESCRIPTION) like "%CHILDREN%" or\
UPPER(DESCRIPTION) like "%JUVENILE%"

#List all kidnapping crimes involving a child?
%sql select * from CHICAGO_CRIME_DATA \
where UPPER(PRIMARY_TYPE) like "%MINOR%" or\
UPPER(PRIMARY_TYPE) like "%CHILDREN%" or\
UPPER(PRIMARY_TYPE) like "%JUVENILE%" or\
UPPER(PRIMARY_TYPE) like "%KIDNAPPED%" \
or UPPER(DESCRIPTION) like "%MINOR" or\
UPPER(DESCRIPTION) like "%CHILDREN%" or\
UPPER(DESCRIPTION) like "%JUVENILE%" or\
UPPER(DESCRIPTION) like "%KIDNAPPING%"

#List the kind of crimes that were recorded at schools. (No repetitions)
%sql select * from CHICAGO_CRIME_DATA \
where UPPER(LOCATION_DESCRIPTION) like "%SCHOOL%"

#List the type of schools along with the average safety score for each type.
%sql select "Elementary, Middle, or High School" as TYPE_OF_SCHOOLS , avg(SAFETY_SCORE)\
from CHICAGO_PUBLIC_SCHOOLS\
group by "Elementary, Middle, or High School"

#List 5 community areas with highest % of households below poverty line
%sql select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc limit 5

#Which community area is most crime prone? Display the coumminty area number only.
%sql select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA \
group by COMMUNITY_AREA_NUMBER \
order by count(*) desc limit 1

#Use a sub-query to find the name of the community area with highest hardship index
%sql select COMMUNITY_AREA_NAME from CENSUS_DATA\
where HARDSHIP_INDEX=\
(select max(HARDSHIP_INDEX) from CENSUS_DATA)

#Use a sub-query to determine the Community Area Name with most number of crimes?
%sql select COMMUNITY_AREA_NAME from CENSUS_DATA \
where COMMUNITY_AREA_NUMBER=\
(select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA\
 group by COMMUNITY_AREA_NUMBER \
 order by count(*) desc limit 1)